<a href="https://colab.research.google.com/github/kdwang1808/BDE-1/blob/main/Lab%201_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BDE - Lab1 - Fall 2021**
> Kaidong Wang - Dec 5, 2021

* 本实验采用LeNet-5网络，在MNIST数字识别任务中对比了使用CPU，单GPU和多GPU并行训练的效率。

## 1. 查看CPU型号与个数

In [ ]:
! cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

## 2. 查看GPU型号与个数

In [ ]:
! nvidia-smi

Sun Dec  5 16:20:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   34C    P0    40W / 300W |      8MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3E:00.0 Off |                    0 |
| N/A   

## 3. 构建LeNet-5网络


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.multiprocessing as mp
import torch.distributed as dist
from tqdm import tqdm
from datetime import datetime

In [ ]:
# Data Preprocess
def get_transform(size, padding, mean, std, preprocess):
    transform = []
    if preprocess:
        transform.append(transforms.RandomCrop(size=size, padding=padding))
        transform.append(transforms.RandomHorizontalFlip())
    transform.append(transforms.ToTensor())
    transform.append(transforms.Normalize(mean, std))
    return transforms.Compose(transform)

In [ ]:
# Model Evaluation
def eval(model, loss, dataloader, device, verbose):
    model.eval()
    total = 0
    correct1 = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            total += loss(output, labels).item() * images.size(0)
            _, pred = output.topk(5, dim=1)
            correct = pred.eq(labels.view(-1, 1).expand_as(pred))
            correct1 += correct[:,:1].sum().item()
    average_loss = total / len(dataloader.dataset)
    accuracy1 = 100. * correct1 / len(dataloader.dataset)
    if verbose:
        print('Evaluation: Average loss: {:.4f}, Top 1 Accuracy: {}/{} ({:.2f}%)'.format(
            average_loss, correct1, len(dataloader.dataset), accuracy1))

In [ ]:
# Build LeNet5
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Prepare Dataset
input_shape, num_classes = (1, 28, 28), 10
mean, std = (0.1307,), (0.3081,)
transform = get_transform(size=28, padding=0, mean=mean, std=std, preprocess=False)

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=True)

## 4. CPU训练

In [ ]:
device = torch.device("cpu")

torch.manual_seed(0)
model = LeNet5().to(device=device)
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)
model.to(device)

epochs = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0)

start = datetime.now()
total_step = len(train_dataloader)
for epoch in tqdm(range(epochs)):
    for i, (images, labels) in enumerate(train_dataloader):
        images, labels= images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f'Epoch: [{epoch + 1}/{epochs}], Step: [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

print("\nTraining time: " + str(datetime.now() - start))
eval(model, criterion, test_dataloader, device, 1)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: [1/30], Step: [100/235], Loss: 2.2846
Epoch: [1/30], Step: [200/235], Loss: 2.2301


  3%|▎         | 1/30 [00:18<08:57, 18.52s/it]

Epoch: [2/30], Step: [100/235], Loss: 1.7188
Epoch: [2/30], Step: [200/235], Loss: 0.6826


  7%|▋         | 2/30 [00:36<08:34, 18.36s/it]

...

Epoch: [29/30], Step: [100/235], Loss: 0.0515
Epoch: [29/30], Step: [200/235], Loss: 0.0570


 97%|█████████▋| 29/30 [09:35<00:20, 20.22s/it]

Epoch: [30/30], Step: [100/235], Loss: 0.0438
Epoch: [30/30], Step: [200/235], Loss: 0.0477


100%|██████████| 30/30 [09:55<00:00, 19.86s/it]



Training time: 0:09:55.736387
Evaluation: Average loss: 0.0609, Top 1 Accuracy: 9810/10000 (98.10%)


## 5. 单GPU训练 (Tesla V100)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
model = LeNet5().to(device=device)
# if torch.cuda.device_count() > 1:
#     model = nn.DataParallel(model)
model.to(device)

epochs = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0)

start = datetime.now()
total_step = len(train_dataloader)
for epoch in tqdm(range(epochs)):
    for i, (images, labels) in enumerate(train_dataloader):
        images, labels= images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f'Epoch: [{epoch + 1}/{epochs}], Step: [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

print("\nTraining time: " + str(datetime.now() - start))
eval(model, criterion, test_dataloader, device, 1)

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /home/mist/pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [1/30], Step: [100/235], Loss: 2.2846
Epoch: [1/30], Step: [200/235], Loss: 2.2301


  3%|▎         | 1/30 [00:11<05:24, 11.18s/it]

Epoch: [2/30], Step: [100/235], Loss: 1.7189
Epoch: [2/30], Step: [200/235], Loss: 0.6827


  7%|▋         | 2/30 [00:20<04:43, 10.11s/it]

...

Epoch: [29/30], Step: [100/235], Loss: 0.0524
Epoch: [29/30], Step: [200/235], Loss: 0.0578


 97%|█████████▋| 29/30 [04:26<00:09,  9.10s/it]

Epoch: [30/30], Step: [100/235], Loss: 0.0438
Epoch: [30/30], Step: [200/235], Loss: 0.0475


100%|██████████| 30/30 [04:35<00:00,  9.18s/it]



Training time: 0:04:35.382380
Evaluation: Average loss: 0.0610, Top 1 Accuracy: 9813/10000 (98.13%)


## 6. 多GPU并行训练 (2*Tesla V100)
> 方法：数据并行(DP)，单进程，多线程，多用于单机多卡

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Prepare Dataset
input_shape, num_classes = (1, 28, 28), 10
mean, std = (0.1307,), (0.3081,)
transform = get_transform(size=28, padding=0, mean=mean, std=std, preprocess=False)

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
model = LeNet5().to(device=device)
if torch.cuda.device_count() > 1:
    print("GPU number: ", torch.cuda.device_count())
    model = nn.DataParallel(model)
model.cuda()

epochs = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0)

start = datetime.now()
total_step = len(train_dataloader)
for epoch in tqdm(range(epochs)):
    for i, (images, labels) in enumerate(train_dataloader):
        images, labels= images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print(f'Epoch: [{epoch + 1}/{epochs}], Step: [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

print("\nTraining time: " + str(datetime.now() - start))
eval(model, criterion, test_dataloader, device, 1)

GPU number:  2


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: [1/30], Step: [100/118], Loss: 2.2792


  3%|▎         | 1/30 [00:10<05:13, 10.81s/it]

Epoch: [2/30], Step: [100/118], Loss: 2.2004


...

Epoch: [29/30], Step: [100/118], Loss: 0.0797


 97%|█████████▋| 29/30 [04:46<00:09,  9.62s/it]

Epoch: [30/30], Step: [100/118], Loss: 0.0865


100%|██████████| 30/30 [04:56<00:00,  9.88s/it]



Training time: 0:04:56.333004
Evaluation: Average loss: 0.0823, Top 1 Accuracy: 9760/10000 (97.60%)


## 7. 效率对比

* CPU训练：9min58s
* 单GPU训练：4min35s
* 多GPU训练：4min56s

**结论：** GPU训练效率远高于CPU，但使用数据并行Data Parallel (DP) 的方法对训练效率没有明显提升，这是由于实验中使用数据量较小，且DP只用一个进程来计算模型权重并在每个批处理期间分发到各GPU，各GPU间通信效率低导致GPU利用率进一步降低。采用分布式数据并行Distributed Data parallel (DDP) 进行多进程训练可进一步提升效率。